In [1]:
import requests
import json

from config import youtube_key


In [8]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.videos.list using API Key
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/code-samples#python

import googleapiclient.discovery
import googleapiclient.errors



In [14]:
API_KEY = youtube_key
CATEGORY_ID = '28'  # DIY category
START_DATE = '2024-01-02T00:00:00Z'
END_DATE = '2024-01-02T23:59:59Z'
BASE_URL = 'https://www.googleapis.com/youtube/v3/search'


In [15]:
# import requests

# API_KEY = youtube_key  # Replace with your API key
# CATEGORY_ID = '28'  # DIY category ID
# START_DATE = '2024-01-01T00:00:00Z'
# END_DATE = '2024-01-1T23:59:59Z'
# BASE_URL = 'https://www.googleapis.com/youtube/v3/search'

def get_video_ids(api_key, category_id, start_date, end_date):
    video_ids = []
    url = f"{BASE_URL}?part=snippet&publishedAfter={start_date}&publishedBefore={end_date}&videoCategoryId={category_id}&type=video&key={api_key}"

    while url:
        response = requests.get(url)
        data = response.json()

        for item in data.get('items', []):
            video_ids.append(item['id']['videoId'])

        # Check for the next page
        url = data.get('nextPageToken')
        if url:
            url = f"{BASE_URL}?part=snippet&publishedAfter={start_date}&publishedBefore={end_date}&videoCategoryId={category_id}&type=video&pageToken={url}&key={api_key}"

    return video_ids



In [16]:

video_ids = get_video_ids(API_KEY, CATEGORY_ID, START_DATE, END_DATE)
print(video_ids)




[]


In [17]:
# -*- coding: utf-8 -*-

# Sample Python code for youtube.videos.list using API Key
# See instructions for running these code samples locally:
# https://developers.google.com/explorer-help/code-samples#python

import googleapiclient.discovery
import googleapiclient.errors
import json


In [20]:
import googleapiclient.discovery
import json

API_KEY = youtube_key  # Replace with your actual API key

def main(video_ids):
    # Set up the YouTube API client
    api_service_name = "youtube"
    api_version = "v3"

    # Build the YouTube API client using the API key
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=API_KEY)

    # # List of video IDs (replace with the actual IDs you need)
    video_ids = [
        "Jig1UqBX3mM",
        "s0XdMjmLVho",
        "fjYGQnYOPvQ",
        "uH4jsSLXjHA",
        "M6qylHuPCeI",
        "WgTDGxV-Zo4",
        "nDLb8_wgX50",
        "KX3GThRK-Fo",
        "sMsGZ6KZa4M",
        "aARNcZ1ooFE"
    ]

    # Make the API request to get video details
    request = youtube.videos().list(
        part="snippet,contentDetails,statistics,status,topicDetails",
        id=",".join(video_ids)  # Join the list of IDs into a comma-separated string
    )
    response = request.execute()

    # Pretty-print the JSON response
    print(json.dumps(response, indent=4))
    filename = 'video_dat.json'
    with open(filename, 'w') as file:
        json.dump(response, file, indent=4)

if __name__ == "__main__":
    main(video_ids)



{
    "kind": "youtube#videoListResponse",
    "etag": "njNdkKI0ezHW6R9cfVU74Y1qIzY",
    "items": [
        {
            "kind": "youtube#video",
            "etag": "w54ZvpI_9ja5ZeCi6qE54y6rp6s",
            "id": "Jig1UqBX3mM",
            "snippet": {
                "publishedAt": "2024-01-01T16:34:47Z",
                "channelId": "UCRFzfDmxPoedpWbO1zqE1yw",
                "title": "Proof For Moon Landing \ud83d\ude44 w/ Neil deGrasse Tyson",
                "description": "For COPYRIGHT ISSUES, please contact us at: (officialuniverselair@gmail.com) This content is shared for awareness purposes only and does not belong to Universe Lair. If you own the rights and want it removed, email us at (officialuniverselair@gmail.com), and we will promptly delete it. This content is used under Section 107 of the Copyright Act 1976, allowing for fair use in criticism, commenting, news reporting, teaching, scholarship, and research. Fair use is permitted for non-profit, educational, or pers